In [1]:
import json
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
class DataLoader:
    def __init__(self,datasetPath,batch,imageSegmentNo) -> None:
        self.datasetPath = datasetPath
        
        self.batchPath = f'{datasetPath}/{batch}'
        self.classes = os.listdir(self.batchPath)


        self.imageSegmentNo = imageSegmentNo

        self.scaledData = []
        

        self.dataExtractor()
        self.shuffler()
        

    def dataExtractor(self):
        for i in range(len(self.classes)):
            if len(self.classes[i].split('.'))<=1:

                for imgdir in os.listdir(f'{self.batchPath}/{self.classes[i]}'):
                    if len(imgdir.split('.'))<=1:
                        jsonMetadata = self.jsonDataLoader(f'{self.batchPath}/{self.classes[i]}/{imgdir}/metadata.json')
                        

                        self.scaledData.append([  self.featureScaler(jsonMetadata['features'][self.imageSegmentNo]), jsonMetadata['classes'][self.imageSegmentNo]])
                        # self.y.append()


                        


    def jsonDataLoader(self,jsonPath):
        with open(jsonPath) as jsonfile:
            currentData = json.load(jsonfile)
            return currentData

    def featureScaler(self,feature):

        maxVal = max(feature[0])
        minVal = min(feature[0])

        dezmin = 0
        dezmax = 1

        scaledFeatures =  []
        for i in feature[0]:
            scaled = (    (   (i - minVal)/(maxVal-minVal)  ) /   (dezmax-dezmin)    ) + dezmin
            scaledFeatures.append(scaled)

        return np.array(scaledFeatures)


    def shuffler(self):
        valData=  np.array(self.scaledData)
        np.random.shuffle(valData)
        
        ListX = []
        Listy = []


        for i in valData:
            ListX.append(i[0])
            Listy.append(i[1])

        self.X = np.array(ListX)
        self.y = np.array(Listy)
        


In [5]:
valDataComplete = []
for i in range(9):
    valDataComplete.append(DataLoader('All_200X','val',i))


/var/folders/4m/_7y4kbtd0l5gjj3fhdysn97c0000gn/T/ipykernel_20966/3157486618.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valData=  np.array(self.scaledData)


In [6]:
trainDataComplete = []
for i in range(9):
    trainDataComplete.append(DataLoader('All_200X','train',i))

/var/folders/4m/_7y4kbtd0l5gjj3fhdysn97c0000gn/T/ipykernel_20966/3157486618.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valData=  np.array(self.scaledData)


In [7]:
testDataComplete = []
for i in range(9):
    testDataComplete.append(DataLoader('All_200X','test',i))

/var/folders/4m/_7y4kbtd0l5gjj3fhdysn97c0000gn/T/ipykernel_20966/3157486618.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valData=  np.array(self.scaledData)


In [8]:
class multiSVM:
    def __init__(self,traindata,testdata) -> None:
        self.clfs = []
        self.traindata = traindata
        self.testdata = testdata
        self.scores = []

    def modelCreator(self):
        for i in range(9):
            self.clfs.append(SVC(kernel='rbf', C=1).fit(self.traindata[i].X,self.traindata[i].y))
    
    def estimator(self):
        for i in range(9):
            y_preds = self.clfs[i].predict(self.testdata[i].X)
            self.scores.append(accuracy_score(self.testdata[i].y,y_preds))

In [9]:
ensembleSVMs = multiSVM(trainDataComplete,testDataComplete)
ensembleSVMs.modelCreator()
ensembleSVMs.estimator()
print(ensembleSVMs.scores)

[0.8360655737704918, 0.8426229508196721, 0.8426229508196721, 0.8557377049180328, 0.8360655737704918, 0.8557377049180328, 0.8426229508196721, 0.8327868852459016, 0.8426229508196721]
